In [ ]:
!pip install -q ipython-sql prettytable

import prettytable
prettytable.__dict__["DEFAULT"] = prettytable.PLAIN_COLUMNS

%reload_ext sql
%sql sqlite://
%sql PRAGMA foreign_keys = ON;

 * sqlite://
Done.


[]

In [ ]:
%%sql

CREATE TABLE clientes (
    id INTEGER PRIMARY KEY,
    nome TEXT,
    cidade TEXT,
    idade INTEGER
);

INSERT INTO clientes (id, nome, cidade, idade) VALUES
(1, 'Ana', 'São Paulo', 28),
(2, 'Bruno', 'Rio de Janeiro', 34),
(3, 'Carlos', 'Belo Horizonte', 22),
(4, 'Daniela', 'São Paulo', 41),
(5, 'Eduarda', 'Curitiba', 25);



 * sqlite://
Done.
5 rows affected.


[]

In [ ]:
%%sql

CREATE TABLE produtos (
  id INTEGER PRIMARY KEY,
  nome TEXT,
  categoria TEXT,
  preco NUMERIC
);

INSERT INTO produtos (id, nome, categoria, preco) VALUES
(1 , 'Notebook', 'Eletrônicos', 3000.0),
(2, 'Celular', 'Eletrônicos', 2000.0),
(3, 'Fone de Ouvido', 'Acessórios', 200.0),
(4, 'Monitor', 'Eletrônicos', 1000.0);

 * sqlite://
Done.
4 rows affected.


[]

In [ ]:
%%sql

CREATE TABLE pedidos (
  id INTEGER PRIMARY KEY,
  cliente_id INTEGER,
  produto_id INTEGER,
  data_pedido DATE,
  quantidade INTEGER,
  FOREIGN KEY (cliente_id) REFERENCES clientes(id),
  FOREIGN KEY (produto_id) REFERENCES produtos(id)
);

INSERT INTO pedidos (id, cliente_id, produto_id, data_pedido, quantidade) VALUES
(1, 1, 1, '2023-01-10', 1),
(2, 2, 2, '2023-01-15', 2),
(3, 1, 3, '2023-02-01', 1),
(4, 3, 1, '2023-02-10', 1),
(5, 2, 3, '2023-03-05', 3),
(6, 4, 4, '2023-03-15', 1),
(7, 1, 2, '2023-03-20', 1);




 * sqlite://
Done.
7 rows affected.


[]

In [ ]:
%%sql
SELECT name FROM sqlite_master
WHERE type = 'table';

 * sqlite://
Done.


name
clientes
produtos
pedidos


In [ ]:
%%sql PRAGMA table_info(clientes);

 * sqlite://
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,nome,TEXT,0,None,0
2,cidade,TEXT,0,None,0
3,idade,INTEGER,0,None,0


In [ ]:
%%sql PRAGMA table_info(produtos)

 * sqlite://
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,nome,TEXT,0,None,0
2,categoria,TEXT,0,None,0
3,preco,NUMERIC,0,None,0


In [ ]:
%%sql PRAGMA table_info(pedidos)

 * sqlite://
Done.


cid,name,type,notnull,dflt_value,pk
0,id,INTEGER,0,None,1
1,cliente_id,INTEGER,0,None,0
2,produto_id,INTEGER,0,None,0
3,data_pedido,DATE,0,None,0
4,quantidade,INTEGER,0,None,0


In [ ]:
# 1.Liste todos os clientes e suas respectivas cidades.
%%sql
SELECT
  nome,
  cidade
FROM
  clientes;

 * sqlite://
Done.


nome,cidade
Ana,São Paulo
Bruno,Rio de Janeiro
Carlos,Belo Horizonte
Daniela,São Paulo
Eduarda,Curitiba


In [ ]:
# 2.Quais clientes fizeram pedidos?
%%sql
SELECT DISTINCT
  c.nome
FROM
  pedidos AS p LEFT JOIN clientes AS c ON
  p.cliente_id = c.id;

 * sqlite://
Done.


nome
Ana
Bruno
Carlos
Daniela


In [ ]:
# 3.Qual foi o total de pedidos por cidade?
%%sql
SELECT
  c.cidade,
  COUNT(p.data_pedido) AS quantidade_de_pedidos
FROM
  clientes AS c LEFT JOIN pedidos AS p ON
  p.cliente_id = c.id
GROUP BY
  c.cidade
ORDER BY
  quantidade_de_pedidos DESC;

 * sqlite://
Done.


cidade,quantidade_de_pedidos
São Paulo,4
Rio de Janeiro,2
Belo Horizonte,1
Curitiba,0


In [ ]:
# 4.Qual produto teve o maior número de pedidos?
%%sql
SELECT
  pr.nome,
  COUNT(pe.data_pedido) AS quantidade_de_pedidos
FROM
  produtos AS pr LEFT JOIN pedidos AS pe ON
  pr.id = pe.produto_id
GROUP BY
  pr.nome
ORDER BY
  quantidade_de_pedidos DESC;

 * sqlite://
Done.


nome,quantidade_de_pedidos
Notebook,2
Fone de Ouvido,2
Celular,2
Monitor,1


In [ ]:
# 5.Qual cliente gastou mais dinheiro no total?
%%sql
SELECT
  c.nome,
  SUM(pr.preco) AS valor_total
FROM
  clientes AS c LEFT JOIN pedidos AS pe ON
  c.id = pe.cliente_id LEFT JOIN produtos AS pr ON
  pe.produto_id = pr.id
GROUP BY
  c.nome
ORDER BY
  valor_total DESC;

 * sqlite://
Done.


nome,valor_total
Ana,5200
Carlos,3000
Bruno,2200
Daniela,1000
Eduarda,None


In [ ]:
# 6.Média de idade dos clientes por cidade.
%%sql
SELECT
  cidade,
  AVG(idade) AS media_de_idade
FROM
  clientes
GROUP BY
  cidade
ORDER BY
  media_de_idade DESC;

 * sqlite://
Done.


cidade,media_de_idade
São Paulo,34.5
Rio de Janeiro,34.0
Curitiba,25.0
Belo Horizonte,22.0


In [ ]:
# 7.Liste os 5 produtos mais vendidos em quantidade.
%%sql
SELECT
  pr.nome,
  COUNT(pe.data_pedido) AS quantidade_de_pedidos
FROM
  produtos AS pr LEFT JOIN pedidos AS pe ON
  pr.id = pe.produto_id
GROUP BY
  pr.nome
ORDER BY
  quantidade_de_pedidos DESC;

 * sqlite://
Done.


nome,quantidade_de_pedidos
Notebook,2
Fone de Ouvido,2
Celular,2
Monitor,1


In [ ]:
# 8.Fature por categoria de produto.
%%sql
SELECT
  pr.categoria,
  pr.preco * COUNT(data_pedido) AS total_categoria
FROM
  produtos AS pr LEFT JOIN pedidos AS pe ON
  pr.id = pe.produto_id
GROUP BY
  pr.categoria
ORDER BY
  total_categoria DESC;

 * sqlite://
Done.


categoria,total_categoria
Eletrônicos,15000
Acessórios,400


In [ ]:
# 9.Clientes que nunca compraram nada.
%%sql
SELECT
  nome
FROM
  clientes
WHERE
  nome NOT IN (SELECT
                  nome
              FROM
                pedidos AS pe LEFT JOIN clientes AS c ON
                pe.cliente_id = c.id);

 * sqlite://
Done.


nome
Eduarda


In [ ]:
# 10.Total de vendas por mês (use strftime('%Y-%m', data_pedido)).
%%sql
SELECT
  strftime('%Y-%m', data_pedido) AS ano_mes,
  SUM(pr.preco) AS total
FROM
  pedidos AS pe LEFT JOIN produtos AS pr ON
  pe.produto_id = pr.id
GROUP BY
  strftime('%Y-%m', data_pedido)
ORDER BY
  total DESC;

 * sqlite://
Done.


ano_mes,total
2023-01,5000
2023-03,3200
2023-02,3200


## 🧠 Desafios Propostos (Treine suas habilidades SQL)

1. Liste todos os clientes e suas respectivas cidades.  
2. Quais clientes fizeram pedidos?  
3. Qual foi o total de pedidos por cidade?  
4. Qual produto teve o maior número de pedidos?  
5. Qual cliente gastou mais dinheiro no total?  
6. Média de idade dos clientes por cidade.  
7. Liste os 5 produtos mais vendidos em quantidade.  
8. Fature por categoria de produto.  
9. Clientes que nunca compraram nada.  
10. Total de vendas por mês (use `strftime('%Y-%m', data_pedido)`).
